In [62]:
import os
import re
import pandas as pd
from tqdm import tqdm

class Zerofucks:
    
    def __init__(self):
        self.df = pd.DataFrame(
            {'file': [], 'line_no': [], 'badword': [], 'content': []}, 
            columns=['file', 'line_no', 'badword', 'content']
        )
    
    def __makeRegex__(self, shitToFind=["fuck", "shit"]):
        '''
        shitToFind is a list of word literals,
        uses word boundary \b

        returns a regex string
        '''
        shitString = ''
        for word in shitToFind:
            # shitString += r"\b" + word + r"\b|"
            
            # case insensitive (?i) has deprecation warning:
            # https://github.com/bottlepy/bottle/issues/949
            shitString += r"(?i)\b" + word + r"\b|"
        return shitString[:-1]
        
    def write_df(self, path='./', filename='out.csv'):
        '''
        path: path to where you want to put the file
        filename: name of file
        
        Writes the csv from self.df to a csv file
        '''
        try:
            self.df.to_csv(path_or_buf=path + filename, index=False, chunksize=1000)
            print("File written to {}".format(path + filename))
        except:
            print("Unable to write csv, did you run .find_shit() yet?")
            
    def erase_shit(self, input_file, output_file, bad_words=['fuck', 'shit']):
        '''
        Erases any bad words from the user specified files.
        '''
        
        print('Replacing bad words in {}...'.format(input_file))
#         try:
        infile = open(input_file, mode='r').readlines()
        outfile = open(output_file, mode='w')
        shit_dict = {}
        for content in tqdm(infile):
            if len(re.findall(self.__makeRegex__(bad_words), content)) > 0:
#                 print('Replacing {}...'.format(re.findall(self.__makeRegex__(bad_words), content)[0]))
                if re.findall(self.__makeRegex__(bad_words), content)[0] in shit_dict:
                    shit_dict[re.findall(self.__makeRegex__(bad_words), content)[0]] += 1
                else:
                    shit_dict[re.findall(self.__makeRegex__(bad_words), content)[0]] = 1
            outfile.write(re.sub(self.__makeRegex__(bad_words), ' ', content))
#         except:
#             print('Unable to replace bad words in that file.')
        return shit_dict
    
    def find_shit(self, root='./', bad_words=['fuck', 'shit']):
        '''
        root: the directory root where you want the crawl to start
        bad_words: list of words you want to search for.
            These can also be regular expressions. It will
            match partials, so 'fuck' will match 'fucking',
            'unfuckingbelievable', etc. Use \b and similar
            to constrain to word boundaries.
        
        The meat and potatoes, this is what conducts the walk
        and writes the resultant dataframe to self.df.
        
        To export the df to a csv, use the write_df() method.
        '''        
        for item in os.walk(root):
            
            # keep records for each directory parsed
            file_df = []
            line_no_df = []
            badword_df = []
            content_df = []
            
            for file in item[2]:
                print('Searching {}'.format(item[0]+'/'+file))
                try:
                    openfile = open(item[0] + '/' + file, "r").readlines()
                    for line_no, content in enumerate(openfile):
                        if len(re.findall(self.__makeRegex__(bad_words), content)) > 0:
                            for i, badword in enumerate(re.findall(self.__makeRegex__(bad_words), content)):
                                file_df.append(item[0]+'/'+file)
                                line_no_df.append(line_no)
                                badword_df.append(re.findall(self.__makeRegex__(bad_words),content)[i])
                                # rstrip to remove newline character
                                content_df.append(content.rstrip())
                except:
                    pass
                            
#             print(content_df)

            # write the records to the dataframe with each dir parsed
            self.df = self.df.append(pd.DataFrame({'file': file_df, 
                            'line_no': line_no_df, 
                            'badword': badword_df,
                            'content': content_df},
                            columns=['file', 'line_no', 'badword', 'content']),
                            ignore_index=True)
        
        # change the silly auto-detected float line_no to an integer
        self.df['line_no'] = self.df.copy()['line_no'].apply(lambda x: int(x))
        # return self.df
        print("Dataframe successfully created, \n \
            use <obj>.df to print the df, or \n \
            .write_df() method to write to file.")

Find words in files

Use this to find bad words in multiple files

In [63]:
# instantiate class
myFinder = Zerofucks()

# load in bad words list from file,
# write to list my_badwords
my_badwords = []
badwords_file = open('./bad_words.txt', "r").readlines()
for line in badwords_file:
    my_badwords.append(line.rstrip())

In [ ]:
# find stuff!
myFinder.find_shit('./course_dump', my_badwords)

In [ ]:
# myFinder.df.iloc[16,:].values
myFinder.df

In [ ]:
myFinder.write_df(filename='output.csv')

Erase badwords from a file and return a 'cleaned' file

Use this to erase badwords from a single file and return a cleaned file

In [66]:
# erase stuff! (returns a dictionary)
out_dict = myFinder.erase_shit('./labeledTrainData.tsv', './out.tsv', my_badwords)
# let's get a report of the stuff we replaced in the erase_shit() call:
out_df = pd.DataFrame.from_dict(out_dict, orient='index')
out_df.columns = ['count']
out_df.sort_values('count', ascending=False)[:10]

  0%|          | 6/24138 [00:00<07:15, 55.38it/s]

Replacing bad words in ./labeledTrainData.tsv...


100%|██████████| 24138/24138 [04:50<00:00, 83.04it/s]


,count
sex,783
crap,606
hell,521
God,440
dog,322
gun,314
god,223
drug,215
damn,186
bloody,172
